# Web Scraping with lazy load

Page to be scraped: El Espectador

In [37]:
# Importing required libraries
import time

from bs4 import BeautifulSoup
from selenium import webdriver

from pymongo import MongoClient

In [38]:
# Creating a connection to MongoDB
client = MongoClient('localhost', 27017)
db = client['news']
collection = db['elespectador']

In [39]:
# Base URL of the site to be analyzed
SITE_URL = 'https://www.elespectador.com'

In [40]:
# Firefox web driver path
# Download the driver for you S.O. here: https://github.com/mozilla/geckodriver/releases
DRIVER_PATH = './geckodriver.exe'

In [41]:
# Creating a new firefox window
browser = webdriver.Firefox(executable_path = DRIVER_PATH)

<ipython-input-41-c8b61cbdd697>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Firefox(executable_path = DRIVER_PATH)


In [42]:
def make_request(browser, categories):
    if (len(categories)<6):
        test = []
        for x in categories:    
            for i in range(5):
                print(i)
                # Making the request and rendering the browser
                if (i==0):
                    browser.get(SITE_URL + x )
                    print(SITE_URL + x)
                else:
                    print(SITE_URL + x + str(i+1)+'/')
                    browser.get(SITE_URL + x + str(i+1)+'/')
                # Simulating vertical scrolling for handling lazy load
                check_height = browser.execute_script('return document.body.scrollHeight;')
                while True:
                    browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
                    time.sleep(3)
                    height = browser.execute_script('return document.body.scrollHeight;')
                    if height == check_height: 
                        break 
                    check_height = height
                test.append(BeautifulSoup(browser.page_source, 'html.parser'))
            # Getting HTML content and passing it to BeautifulSoup for scraping analysis
        return test
    else:
        # Making the request and rendering the browser
        browser.get(SITE_URL + categories)

        # Simulating vertical scrolling for handling lazy load
        check_height = browser.execute_script('return document.body.scrollHeight;')
        while True:
            browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            time.sleep(3)
            height = browser.execute_script('return document.body.scrollHeight;')
            if height == check_height: 
                break 
            check_height = height

        # Getting HTML content and passing it to BeautifulSoup for scraping analysis
        return BeautifulSoup(browser.page_source, 'html.parser')

In [43]:
# Getting HTML content for a particular news listing page
categories = ['/archivo/judicial/','/archivo/politica/','/archivo/colombia/','/archivo/economia/','/archivo/bogota/']
soup = make_request(browser, categories)

0
https://www.elespectador.com/archivo/judicial/
1
https://www.elespectador.com/archivo/judicial/2/
2
https://www.elespectador.com/archivo/judicial/3/
3
https://www.elespectador.com/archivo/judicial/4/
4
https://www.elespectador.com/archivo/judicial/5/
0
https://www.elespectador.com/archivo/politica/
1
https://www.elespectador.com/archivo/politica/2/
2
https://www.elespectador.com/archivo/politica/3/
3
https://www.elespectador.com/archivo/politica/4/
4
https://www.elespectador.com/archivo/politica/5/
0
https://www.elespectador.com/archivo/colombia/
1
https://www.elespectador.com/archivo/colombia/2/
2
https://www.elespectador.com/archivo/colombia/3/
3
https://www.elespectador.com/archivo/colombia/4/
4
https://www.elespectador.com/archivo/colombia/5/
0
https://www.elespectador.com/archivo/economia/
1
https://www.elespectador.com/archivo/economia/2/
2
https://www.elespectador.com/archivo/economia/3/
3
https://www.elespectador.com/archivo/economia/4/
4
https://www.elespectador.com/archivo/

In [44]:
# Finding the section where news are contained 
layout = []
for i in range(len(soup)):
    layout.append(soup[i].find(class_ = 'Layout-flexAds'))
print(len(layout))


25


In [45]:
# Getting blocks from layout
blocks = []
for i in range(len(layout)):
    blocks.append(layout[i].find_all(class_ = 'Container Block', recursive = True))
print(len(blocks))

25


In [46]:
# Finding and concatenating news cards
cards = []
print(i)
for i in range(len(blocks)):
    cards.append(blocks[i][0].find_all(class_ = 'Card_rowCardLeft') + blocks[i][1].find_all(class_ = 'Card_rowCardLeft'))
print(len(cards))

24
25


In [47]:
# Building a list with title and relative path of the news founded

news = []
for i in range(len(cards)):
    for x in range(len(cards[i])):         
        news.append({
            'title': cards[i][x].find('h2', class_ = 'Card-Title').find('a').get_text(),
            'relative_path': cards[i][x].find('h2', class_ = 'Card-Title').find('a')['href']
        })
print(len(news))

280


In [48]:
news

[{'title': 'La inteligencia ilegal a defensores de derechos será juzgada por la CorteIDH',
  'relative_path': '/judicial/la-vigilancia-e-inteligencia-ilegal-a-defensores-de-derechos-sera-juzgada-por-cidh/'},
 {'title': '“Artemisa no va a poder frenar la deforestación”: comandante fuerza de tarea Omega',
  'relative_path': '/judicial/artemisa-no-va-a-poder-frenar-la-deforestacion-comandante-fuerza-de-tarea-omega/'},
 {'title': 'Cinco mujeres denuncian abuso sexual en la banda bogotana Electric Mistakes',
  'relative_path': '/judicial/cinco-mujeres-denuncian-abuso-sexual-en-la-banda-electric-mistakes/'},
 {'title': '“Artemisa es una política de golpes de opinión”: Estefanía Ciro, de A la Orilla del Río',
  'relative_path': '/ambiente/amazonas/artemisa-es-una-politica-de-golpes-de-opinion/'},
 {'title': 'Capturan a “Orlando La Muerte”, señalado líder de las disidencias en Arauca',
  'relative_path': '/judicial/capturan-a-orlando-la-muerte-senalado-lider-de-las-disidencias-en-arauca/'},
 {

In [49]:
for n in news:
    # Getting HTML content for each news page
    print(n['relative_path'])
    soup = make_request(browser, n['relative_path'])
    
    # Extracting news metadata
    n['datetime'] = soup.find(class_ = 'ArticleHeader-Date').get_text()
    #n['author'] = soup.find(class_ = 'ACredit-Author').find('a').get_text()
    n['summary'] = soup.find(class_ = 'ArticleHeader-Hook').find('div').get_text()
    
    # Extracting and concatenating news full text
    paragraphs = soup.find_all(class_ = 'font--secondary')
    n['full_text'] = ' '.join([p.get_text() for p in paragraphs])

/judicial/la-vigilancia-e-inteligencia-ilegal-a-defensores-de-derechos-sera-juzgada-por-cidh/
/judicial/artemisa-no-va-a-poder-frenar-la-deforestacion-comandante-fuerza-de-tarea-omega/
/judicial/cinco-mujeres-denuncian-abuso-sexual-en-la-banda-electric-mistakes/
/ambiente/amazonas/artemisa-es-una-politica-de-golpes-de-opinion/
/judicial/capturan-a-orlando-la-muerte-senalado-lider-de-las-disidencias-en-arauca/
/judicial/fiscalia-no-investiga-violencia-hacia-reclusos-denuncia-informe-en-la-corte-constitucional/
/judicial/la-respuesta-militar-del-gobierno-a-un-problema-ambiental/
/colombia/violencia-destruyo-promesa-de-paz-para-muchos-colombianos-consejo-noruego/
/judicial/bienes-por-1-billon-que-serian-del-clan-del-golfo-a-extincion-de-dominio/
/judicial/ataque-con-explosivos-en-nordeste-de-antioquia-deja-tres-militares-muertos/
/judicial/dilan-cruz-y-nicolas-neira-las-calles-que-cuentan-sus-historias/
/judicial/a-la-linea-negra-aun-le-queda-un-pendiente-en-el-consejo-de-estado/
/judicia

AttributeError: 'NoneType' object has no attribute 'get_text'

In [50]:
news

[{'title': 'La inteligencia ilegal a defensores de derechos será juzgada por la CorteIDH',
  'relative_path': '/judicial/la-vigilancia-e-inteligencia-ilegal-a-defensores-de-derechos-sera-juzgada-por-cidh/',
  'datetime': '11 abr. 2022  - 10:47 a.\xa0m.',
  'summary': 'El Colectivo de Abogados José Alvear y el Centro por la Justicia y el Derecho Internacional llevaron ante el órgano internacional las estigmatizaciones a los que se enfrentan los defensores de derechos humanos en el país. En dos audiencias hablarán sobre amenazas, hostigamientos y exilios.',
  'full_text': 'A través de una resolución de 39 páginas, la Corte Interamericana de Derechos Humanos (CorteIDH) le dio el sí a Colombia para juzgar, por primera vez, la vigilancia e inteligencia ilegal a defensores de derechos humanos. El tema llegó al organismo internacional por cuenta de una serie de denuncias que hizo el Colectivo de Abogados José Alvear (Cajar) junto con el Centro por la Justicia y el Derecho Internacional (Cejil

In [52]:
# Storing extracted information for further analysis
collection.insert_many(news)